In [1]:
# Author: Ahbaid Gaffoor
# Date: Wednesday 26th April 2017

## Weather Data with NumPy
1. Load NOAAA station and temperature data from text files
2. Integrate, smooth and plot data
3. Compute daily records
4. Challenge: Compare warmest year of cold location with coldest year of warm location.

## Load NOAA station and temperature data from text files
1. Download file over ftp from http://www.ncdc.noaa.gov
2. Parse a space-separated text file into a Python dictionary.

In [9]:
import numpy as np
import matplotlib.pyplot as pp

# seaborn is an extension to matplotlib to improve ddefault plot formatting and adds additional plot types
import seaborn

In [10]:
# Instruct pyplot to keep plots inline with the notebooks
%matplotlib inline

In [17]:
# How to download files using python
import urllib.request as urlreq
urlreq.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt','stations.txt')

('stations.txt', <email.message.Message at 0x114474ef0>)

In [19]:
# Sample the first 10 lines of stations.txt
# Observations:
# 1. station code 
# 2. longitude and latitude
# 3. sea level in meters
# 4. a name
# 5. Some stations are tagged with "GSN" - GCOS Surface Network
#
# Our focus will be on stations with the GSN tag
#
open('stations.txt','r').readlines()[:10]

['ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       \n',
 'ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    \n',
 'AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196\n',
 'AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194\n',
 'AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217\n',
 'AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218\n',
 'AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930\n',
 'AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938\n',
 'AFM00040948  34.5660   69.2120 1791.3    KABUL INTL                             40948\n',
 'AFM00040990  31.5000   65.8500 1010.0    KANDAHAR AIRPORT                       40990\n']

In [21]:
# Read all lines, skip those that do not have the GSN flag
stations = {}
for line in open('stations.txt','r'):
    if 'GSN' in line:
        fields = line.split()
        # Grab from station name (5th field) onwards separate by ' ' and concatenate into value of dictionary
        # Key name is the station code
        stations[fields[0]] = ' '.join(fields[4:])

In [26]:
len(stations)

994

In [38]:
# Write a function that finds station codes by pattern name
def findstation(s):
    found = {code: name for code, name in stations.items() if s in name}
    print (found)

In [39]:
findstation('LIHUE')

{'USW00022536': 'HI LIHUE WSO AP 1020.1 GSN 91165'}


In [44]:
findstation('SAN')

{'AR000087623': 'SANTA ROSA AERO GSN 87623', 'BLM00085207': 'SAN IGNACIO VELASCO GSN 85207', 'CO000080001': 'SAN ANDRES (ISLA)/S GSN 80001', 'CUM00078310': 'CABO SAN ANTONIO PINAR DEL GSN 78310', 'ECM00084008': 'SAN CRISTOBAL GSN 84008', 'FKM00088889': 'MOUNT PLEASANT GSN 88889', 'ID000096925': 'SANGKAPURA (BAWEAN GSN 96925', 'IV000065599': 'SASSANDRA GSN 65599', 'JM000078388': 'MONTEGO BAY/SANGSTE GSN 78388', 'MG000044259': 'CHOIBALSAN GSN 44259', 'MP000061990': 'PLAISANCE (MAURITIU GSN 61990', 'MY000096491': 'SANDAKAN GSN 96491', 'NH000091554': 'PEKOA AIRPORT (SANT GSN 91554', 'RP000098851': 'GEN. SANTOS GSN 98851', 'RQW00011641': 'PR SAN JUAN L M MARIN AP GSN 78526', 'RSM00023955': 'ALEKSANDROVSKOE GSN 23955', 'RSM00032061': 'ALEKSANDROVSK-SAHALINSKIJ GSN 32061', 'SP000008027': 'SAN SEBASTIAN - IGUELDO GSN 08027', 'SWM00002589': 'GOTSKA SANDON A GSN 02589', 'TOM00065352': 'MANGO/SANSANNE GSN 65352', 'TX000038915': 'CARSANGA GSN 38915', 'USW00012921': 'TX SAN ANTONIO INTL AP GSN HCN 

In [41]:
findstation('SAN DIEGO')

{'USW00023188': 'CA SAN DIEGO LINDBERGH FLD GSN 72290'}


In [45]:
findstation('IRKUTSK')

{'RSM00030710': 'IRKUTSK GSN 30710'}


In [46]:
findstation('MINNEAPOLIS')

{'USW00014922': 'MN MINNEAPOLIS/ST PAUL AP GSN HCN 72658'}


In [52]:
# We will look at data from the 4 stations from LIHUE, SAN DIEGO, IRKUTSK and MINNEAPOLIS
datastations = ['USW00022536','USW00023188','RSM00030710','MINNEAPOLIS']

## Loading Temperature Data
1. Parsing a fixed-field text file using np.genfromtxt
2. Using ranges of NumPy datetime objects

In [62]:
# Files we will be working with for each datastation are:
# ./RSM00030710.dly for IRKUTSK
# ./USW00014922.dly for MINNEAPOLIS
# ./USW00022536.dly for LIHUE
# ./USW00023188.dly for SAN DIEGO

### FORMAT OF DATA FILES (".dly" FILES)

Source: ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt 

Each ".dly" file contains data for one station.  The name of the file
corresponds to a station's identification code.  For example, "USC00026481.dly"
contains the data for the station with the identification code USC00026481).

Each record in a file contains one month of daily data.  The variables on each
line include the following:
```
------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
YEAR         12-15   Integer
MONTH        16-17   Integer
ELEMENT      18-21   Character
VALUE1       22-26   Integer
MFLAG1       27-27   Character
QFLAG1       28-28   Character
SFLAG1       29-29   Character
VALUE2       30-34   Integer
MFLAG2       35-35   Character
QFLAG2       36-36   Character
SFLAG2       37-37   Character
  .           .          .
  .           .          .
  .           .          .
VALUE31    262-266   Integer
MFLAG31    267-267   Character
QFLAG31    268-268   Character
SFLAG31    269-269   Character
------------------------------
```

In [64]:
# LIHUE - USW00022536
# ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt section 

open('USW00022536.dly','r').readlines()[:5]

['USW00022536195002TMAX  256  0  256  0  256  0  267  0  217  0  228  0  256  0  272  0  256  0  256  0  256  0  244  0  256  0  256  0  244  0  244  0  250  0  256  0  239  0  250  0  256  0  256  0  267  0  261  0  267  0  267  0  261  0  261  0-9999   -9999   -9999   \n',
 'USW00022536195002TMIN  178  0  156  0  161  0  167  0  167  0  167  0  189  0  211  0  206  0  217  0  217  0  211  0  200  0  200  0  206  0  183  0  206  0  206  0  206  0  194  0  206  0  200  0  206  0  200  0  211  0  183  0  172  0  200  0-9999   -9999   -9999   \n',
 'USW00022536195002PRCP    0  0    0  0    0  0    0  0  737  0  406  0   36  0   38  0    0T 0    0T 0    0  0    0T 0   18  0    5  0   10  0   18  0   15  0    5  0    0T 0    0T 0   23  0   10  0    3  0   48  0    0T 0    0T 0    0T 0    5  0-9999   -9999   -9999   \n',
 'USW00022536195002SNOW    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0  0    0

In [65]:
open('readme.txt','r').readlines()[98:121]

['------------------------------\n',
 'Variable   Columns   Type\n',
 '------------------------------\n',
 'ID            1-11   Character\n',
 'YEAR         12-15   Integer\n',
 'MONTH        16-17   Integer\n',
 'ELEMENT      18-21   Character\n',
 'VALUE1       22-26   Integer\n',
 'MFLAG1       27-27   Character\n',
 'QFLAG1       28-28   Character\n',
 'SFLAG1       29-29   Character\n',
 'VALUE2       30-34   Integer\n',
 'MFLAG2       35-35   Character\n',
 'QFLAG2       36-36   Character\n',
 'SFLAG2       37-37   Character\n',
 '  .           .          .\n',
 '  .           .          .\n',
 '  .           .          .\n',
 'VALUE31    262-266   Integer\n',
 'MFLAG31    267-267   Character\n',
 'QFLAG31    268-268   Character\n',
 'SFLAG31    269-269   Character\n',
 '------------------------------\n']

** Create a function using np.genfrom text to return record data. **

NumPy needs to be told:
1. The file name 
2. The size of all the fields
3. Which columns we wish to keep 
4. The type of all the fields 
5. Last the names that we want to give to all the fields

In [66]:
def parsefile(filename):
    return np.genfromtxt(
        filename,
        delimiter = dly_delimiter,
        usecols = dly_usecols,
        dtype = dly_dtype,
        names = dly_names
    )

In [68]:
# 11, 4, 2, 4 maps to ID, YEAR, MONTH, ELEMENT
# 5, 1, 1 maps to each value and it's three flags which repeat 31 times -> VALUE1(5) + 3 flags MFLAG1, QFLAG1, SFLAG1
dly_delimiter = [11,4,2,4] + [5,1,1,1] * 31

# We will not use all the columns, we will use year, month, element, or cols 1,2,3 from the delimeter spec
# and the value columns 4 through 31 
dly_usecols = [1,2,3] + [4*i for i in range(1,32)]

# Field types
dly_dtype = [np.int32, np.int32, (np.str,4)] + [np.int32] * 31

# Field names. obs = observable
dly_names = ['year','month','obs'] + [str(day) for day in range(1,31+1)]

In [69]:
lihue=parsefile('USW00022536.dly')

In [70]:
lihue

array([ (1950, 2, 'TMAX', 256, 256, 256, 267, 217, 228, 256, 272, 256, 256, 256, 244, 256, 256, 244, 244, 250, 256, 239, 250, 256, 256, 267, 261, 267, 267, 261, 261, -9999, -9999, -9999),
       (1950, 2, 'TMIN', 178, 156, 161, 167, 167, 167, 189, 211, 206, 217, 217, 211, 200, 200, 206, 183, 206, 206, 206, 194, 206, 200, 206, 200, 211, 183, 172, 200, -9999, -9999, -9999),
       (1950, 2, 'PRCP', 0, 0, 0, 0, 737, 406, 36, 38, 0, 0, 0, 0, 18, 5, 10, 18, 15, 5, 0, 0, 23, 10, 3, 48, 0, 0, 0, 5, -9999, -9999, -9999),
       ...,
       (2015, 9, 'WT03', -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, 1, -9999, 1, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999),
       (2015, 9, 'WT08', -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, 1, -9999, -9999, -9999, 1, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, 1, 1, -9999, -9999, -9999, -9999, -9999, -9999),
       

** IMPROVEMENTS: **

1. We have a complex NumPy record array with all the information contained in the original file. We need to massage this data into a better form. 
2. The temperatures for all the days of the month sit on the same row, which is inconvienient because different months have different numbers of days. Instead, each day should have a separate row. 
3. We also want to associate each data point with a proper NumPy datetime object.

We will first create a function to do these transformations.

In [78]:
def unroll(record):
    '''
    1. We begin by creating a range of dates that correspondes to a row. 
    2. We specify the beginning of the month by including only the year and the month in a string fed to the NumPy 
       datetime 64 object. 
    3. We then create a range of dates using NumPy arange starting at the start date, ending at the start date 
       plus one month. And, with a step of one day.
    '''
    startdate = np.datetime64('{}-{:02}'.format(record['year'],record['month']))
    dates = np.arange(startdate, startdate + np.timedelta64(1,'M'), np.timedelta64(1,'D'))
    
    '''
    Next, we are going to collect the data for the days from the record. 
    We need to specify the field for which we're going to extract the data. 
    We can do that by enclosing dates in enumerate by looping over an index and a date at the same time. 
    And, by using that index to build a name of the record called. 
    Last, divide each data point by ten since temperatures are specified in tens of degrees.
    '''
    rows = [(date,record[str(i+1)]/10) for i,date in enumerate(dates)]
    return rows

In [79]:
lihue[0]

(1950, 2, 'TMAX', 256, 256, 256, 267, 217, 228, 256, 272, 256, 256, 256, 244, 256, 256, 244, 244, 250, 256, 239, 250, 256, 256, 267, 261, 267, 267, 261, 261, -9999, -9999, -9999)

In [80]:
unroll(lihue[0])

[(numpy.datetime64('1950-02-01'), 25.600000000000001),
 (numpy.datetime64('1950-02-02'), 25.600000000000001),
 (numpy.datetime64('1950-02-03'), 25.600000000000001),
 (numpy.datetime64('1950-02-04'), 26.699999999999999),
 (numpy.datetime64('1950-02-05'), 21.699999999999999),
 (numpy.datetime64('1950-02-06'), 22.800000000000001),
 (numpy.datetime64('1950-02-07'), 25.600000000000001),
 (numpy.datetime64('1950-02-08'), 27.199999999999999),
 (numpy.datetime64('1950-02-09'), 25.600000000000001),
 (numpy.datetime64('1950-02-10'), 25.600000000000001),
 (numpy.datetime64('1950-02-11'), 25.600000000000001),
 (numpy.datetime64('1950-02-12'), 24.399999999999999),
 (numpy.datetime64('1950-02-13'), 25.600000000000001),
 (numpy.datetime64('1950-02-14'), 25.600000000000001),
 (numpy.datetime64('1950-02-15'), 24.399999999999999),
 (numpy.datetime64('1950-02-16'), 24.399999999999999),
 (numpy.datetime64('1950-02-17'), 25.0),
 (numpy.datetime64('1950-02-18'), 25.600000000000001),
 (numpy.datetime64('1950

In [81]:
def unroll(record):
    '''
    NumPy arange correctly returned the days of February. Instead of a list of tuples, we want to return a 
    proper NumPy record array we can modify the function to do that. 
    
    We'll feed the list rolls to NumPy array and specify the property type. 
    
    It's going to be the date in days followed by the value as a double procedure number.
    '''
    startdate = np.datetime64('{}-{:02}'.format(record['year'],record['month']))
    dates = np.arange(startdate, startdate + np.timedelta64(1,'M'), np.timedelta64(1,'D'))
    
   
    rows = [(date,record[str(i+1)]/10) for i,date in enumerate(dates)]
    #return rows
    return np.array(rows,dtype=[('date','M8[D]'), ('value','d')])

In [82]:
unroll(lihue[0])

array([(datetime.date(1950, 2, 1), 25.6),
       (datetime.date(1950, 2, 2), 25.6),
       (datetime.date(1950, 2, 3), 25.6),
       (datetime.date(1950, 2, 4), 26.7),
       (datetime.date(1950, 2, 5), 21.7),
       (datetime.date(1950, 2, 6), 22.8),
       (datetime.date(1950, 2, 7), 25.6),
       (datetime.date(1950, 2, 8), 27.2),
       (datetime.date(1950, 2, 9), 25.6),
       (datetime.date(1950, 2, 10), 25.6),
       (datetime.date(1950, 2, 11), 25.6),
       (datetime.date(1950, 2, 12), 24.4),
       (datetime.date(1950, 2, 13), 25.6),
       (datetime.date(1950, 2, 14), 25.6),
       (datetime.date(1950, 2, 15), 24.4),
       (datetime.date(1950, 2, 16), 24.4),
       (datetime.date(1950, 2, 17), 25.0),
       (datetime.date(1950, 2, 18), 25.6),
       (datetime.date(1950, 2, 19), 23.9),
       (datetime.date(1950, 2, 20), 25.0),
       (datetime.date(1950, 2, 21), 25.6),
       (datetime.date(1950, 2, 22), 25.6),
       (datetime.date(1950, 2, 23), 26.7),
       (datetime.dat

Last, we want to take all the months contained in a file and concatenate them together into a single NumPy record array. We also want to select a single observable, such as minimum temperature. For this we write a function getobs for get observable. We write a list comprehension containing the unrolled rows for each row of the parsed file. And, select only those that contain our desired observable and we'd feed this list to NumPy concatenate.

In [83]:
def getobs(filename,obs):
    return np.concatenate([unroll(row) for row in parsefile(filename) if row[2] == obs])

In [84]:
getobs('USW00022536.dly','TMIN')

array([(datetime.date(1950, 2, 1), 17.8),
       (datetime.date(1950, 2, 2), 15.6),
       (datetime.date(1950, 2, 3), 16.1), ...,
       (datetime.date(2015, 9, 28), -999.9),
       (datetime.date(2015, 9, 29), -999.9),
       (datetime.date(2015, 9, 30), -999.9)], 
      dtype=[('date', '<M8[D]'), ('value', '<f8')])

In [85]:
getobs('USW00022536.dly','TMAX')

array([(datetime.date(1950, 2, 1), 25.6),
       (datetime.date(1950, 2, 2), 25.6),
       (datetime.date(1950, 2, 3), 25.6), ...,
       (datetime.date(2015, 9, 28), -999.9),
       (datetime.date(2015, 9, 29), -999.9),
       (datetime.date(2015, 9, 30), -999.9)], 
      dtype=[('date', '<M8[D]'), ('value', '<f8')])

In [86]:
getobs('USW00022536.dly','PRCP')

array([(datetime.date(1950, 2, 1), 0.0), (datetime.date(1950, 2, 2), 0.0),
       (datetime.date(1950, 2, 3), 0.0), ...,
       (datetime.date(2015, 9, 28), -999.9),
       (datetime.date(2015, 9, 29), -999.9),
       (datetime.date(2015, 9, 30), -999.9)], 
      dtype=[('date', '<M8[D]'), ('value', '<f8')])